In [4]:
from parsing import load_problem, load_solution
from graph_representation import Graph
import time
from search_methods import search_with_lower_bound

In [5]:
# weryfikuje poprawność sortowania
def verify_toposort(T, G):
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [ ]:
n, m, problem = load_problem("instances/Barnes_setb4x.fjs")
solution = load_solution("solutions/Barnes_setb4x.txt")

t0 = time.time()
new_sol, t = search_with_lower_bound(n, m, problem, solution, 10)
print t, time.time()-t0

((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)
((9, 10, 1), 9, 1)


In [3]:
n, m, problem = load_problem("instances/Barnes_mt10c1.fjs")
solution = load_solution("solutions/Barnes_mt10c1.txt")

G = Graph(n, m, problem, solution)
G.graph[(3,1,4)].successors
G.topological_sort()

In [4]:
G.longest_path_length((0,9,1), (0,1,1))

In [5]:
N = G.generate_neighborhood()

In [26]:
N

[((0, 9, 1), 0, 0),
 ((0, 9, 1), 0, 1),
 ((0, 9, 1), 0, 2),
 ((0, 9, 1), 0, 3),
 ((0, 9, 1), 0, 4),
 ((0, 9, 1), 10, 0),
 ((0, 9, 1), 10, 1),
 ((0, 9, 1), 10, 2),
 ((0, 9, 1), 10, 3),
 ((0, 9, 1), 10, 4),
 ((0, 2, 1), 0, 0),
 ((0, 2, 1), 0, 1),
 ((0, 2, 1), 0, 2),
 ((0, 2, 1), 0, 3),
 ((0, 2, 1), 0, 4),
 ((0, 2, 1), 0, 5),
 ((0, 2, 1), 10, 0),
 ((0, 2, 1), 10, 1),
 ((0, 2, 1), 10, 2),
 ((0, 2, 1), 10, 3),
 ((0, 2, 1), 10, 4),
 ((0, 7, 2), 0, 0),
 ((0, 7, 2), 0, 1),
 ((0, 7, 2), 0, 2),
 ((0, 7, 2), 0, 3),
 ((0, 7, 2), 0, 4),
 ((0, 7, 2), 0, 5),
 ((0, 7, 2), 10, 0),
 ((0, 7, 2), 10, 1),
 ((0, 7, 2), 10, 2),
 ((0, 7, 2), 10, 3),
 ((0, 7, 2), 10, 4),
 ((0, 10, 2), 0, 0),
 ((0, 10, 2), 0, 1),
 ((0, 10, 2), 0, 2),
 ((0, 10, 2), 0, 3),
 ((0, 10, 2), 0, 4),
 ((0, 10, 2), 0, 5),
 ((0, 10, 2), 10, 0),
 ((0, 10, 2), 10, 1),
 ((0, 10, 2), 10, 2),
 ((0, 10, 2), 10, 3),
 ((0, 10, 2), 10, 4),
 ((0, 5, 2), 0, 0),
 ((0, 5, 2), 0, 1),
 ((0, 5, 2), 0, 2),
 ((0, 5, 2), 0, 3),
 ((0, 5, 2), 0, 4),
 ((0, 5, 

In [6]:
G.make_a_move(((4,10,9), 4, 9))

In [8]:
G.graph[(10,10,9)].successors

{(10, 10, 10), (11, 4, 4)}

In [27]:
t0=time.time()
for x in N:
    print x
    print G.lower_bound(x)
time.time()-t0

((0, 9, 1), 0, 0)
839
((0, 9, 1), 0, 1)
839
((0, 9, 1), 0, 2)
839
((0, 9, 1), 0, 3)
839
((0, 9, 1), 0, 4)
839
((0, 9, 1), 10, 0)
839
((0, 9, 1), 10, 1)
839
((0, 9, 1), 10, 2)
904
((0, 9, 1), 10, 3)
1079
((0, 9, 1), 10, 4)
1108
((0, 2, 1), 0, 0)
882
((0, 2, 1), 0, 1)
763
((0, 2, 1), 0, 2)
763
((0, 2, 1), 0, 3)
763
((0, 2, 1), 0, 4)
763
((0, 2, 1), 0, 5)
950
((0, 2, 1), 10, 0)
773
((0, 2, 1), 10, 1)
763
((0, 2, 1), 10, 2)
802
((0, 2, 1), 10, 3)
1003
((0, 2, 1), 10, 4)
1032
((0, 7, 2), 0, 0)
876
((0, 7, 2), 0, 1)
800
((0, 7, 2), 0, 2)
720
((0, 7, 2), 0, 3)
720
((0, 7, 2), 0, 4)
720
((0, 7, 2), 0, 5)
907
((0, 7, 2), 10, 0)
767
((0, 7, 2), 10, 1)
720
((0, 7, 2), 10, 2)
759
((0, 7, 2), 10, 3)
960
((0, 7, 2), 10, 4)
989
((0, 10, 2), 0, 0)
852
((0, 10, 2), 0, 1)
776
((0, 10, 2), 0, 2)
733
((0, 10, 2), 0, 3)
648
((0, 10, 2), 0, 4)
648
((0, 10, 2), 0, 5)
870
((0, 10, 2), 10, 0)
743
((0, 10, 2), 10, 1)
648
((0, 10, 2), 10, 2)
722
((0, 10, 2), 10, 3)
888
((0, 10, 2), 10, 4)
917
((0, 5, 2), 0, 0)
8

3.1341309547424316